GloVe as a weak labeling function
------
Perhaps one of the simplest labeling functions that involves only converting the biased word into a glove vector and classifying that embedding. For a given sentence, we again have to first extract what the first-predicted biased word is. We know what this word will be, since our dataset contains the ground-truth labels for which words were edited for bias. We can thus extract the index of the first biased word, and then do some GloVe embedding featurization based on that particular word. 

In [31]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
params = Params.read_params("experiment_params.json")

In [33]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)

02/26/2020 14:19:14 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
386it [00:00, 5094.18it/s]


In [34]:
from src.utils.weak_labeling_utils import get_glove_features

In [48]:
glove_embeddings = get_glove_features(params.dataset, dataset)

02/26/2020 14:54:05 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


KeyError: 11

In [8]:
dataset.add_data(glove_embeddings, "glove_embeddings")

### This is where the classification experiment starts

In [9]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [10]:
from src.utils.classification_utils import run_bootstrapping

In [11]:
marta_features.shape

torch.Size([324, 90])

In [16]:
statistics = run_bootstrapping(classification_experiment, dataset, params.final_task, num_bootstrap_iters=3, input_key='marta_features', label_key='bias_label', threshold=0.42)

In [17]:
statistics

{'auc': [(0.9503368677016261, 0.9708432171569927), 0.959598823295552],
 'accuracy': [(0.8969600340136054, 0.9377551020408164), 0.9176587301587302]}